In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataloader import create_data_loader,WellDataset
from model import TransformerModel, TransformerModel_ver2
from train import train_model



# Dataloader

In [37]:
dataset = WellDataset(
        csv_path='E:/DS/data/siam/deb/tg.csv',
        parquet_path='E:/DS/data/siam/deb/data.parquet',
        max_seq_len=20
    )

Preprocessing time series data...


100%|██████████████████████████████████████████████████████████████████████████████| 996/996 [00:00<00:00, 2131.08it/s]

Preprocessed 996 time series


In [38]:
dataset.label_dict['7035127a-7d2c-4df2-8731-dfcf69eff50f']

{'binary': array([0, 0, 1, 1, 0, 1, 0, 0], dtype=object),
 'regression': array([nan, 1.013299184, 2.102156549, nan, 1.10329706, nan, nan],
       dtype=object),
 'regression_mask': array([False,  True,  True, False,  True, False, False])}

In [39]:
dataset.label_dict['7035127a-7d2c-4df2-8731-dfcf69eff50f']['binary']

array([0, 0, 1, 1, 0, 1, 0, 0], dtype=object)

In [ ]:
   timeseries_df: pd.DataFrame,
    binary_targets_df = 'E:/DS/data/siam/deb/tg.csv',
    regression_targets_df: pd.DataFrame,
    sequence_length: int = 1000,
    stride: int = 100,
    transform: Optional[Callable] = None,
    batch_size: int = 32,
    shuffle: bool = True,
    num_workers: int = 4

In [41]:
# Create data loaders
train_loader = create_data_loader(
    csv_path='E:/DS/data/siam/deb/tg.csv',
    parquet_path='E:/DS/data/siam/deb/data.parquet',
    batch_size=32,
    max_seq_len=512,
    shuffle = False,
    num_workers = 0,
)

Preprocessing time series data...


100%|██████████████████████████████████████████████████████████████████████████████| 996/996 [00:00<00:00, 1768.49it/s]


Preprocessed 996 time series


In [45]:
import pandas as pd

In [46]:
ch = pd.read_csv('E:/DS/data/siam/deb/tg.csv')

In [47]:
ch2 = pd.read_parquet('E:/DS/data/siam/deb/data.parquet')

In [50]:
ch.file_name.nunique()

1000

In [48]:
ch2[ch2.file_name=='af3cb4de-72a7-4a55-b288-04f741eb2138']

,file_name,time,atm,der_atm


In [6]:
b = next(iter(dataset))

In [42]:
batch = next(iter(train_loader))

In [43]:
batch

{'input_features': tensor([[[8.3333e-03, 1.6684e+01, 1.0773e+01],
          [1.6667e-02, 2.6314e+01, 1.7047e+01],
          [2.5000e-02, 3.3954e+01, 2.0997e+01],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00]],
 
         [[8.3333e-03, 4.2215e+04, 8.4655e+04],
          [1.6667e-02, 8.5542e+04, 4.0427e+04],
          [2.5000e-02, 9.6681e+04, 2.5501e+04],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00]],
 
         [[1.6667e-02, 2.7870e-02, 2.1391e-02],
          [3.3333e-02, 5.5401e-02, 5.8148e-02],
          [5.0000e-02, 8.3271e-02, 9.9316e-02],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00]],
 
         ...,
 
         [[1.4681e-02, 8.1104e-01, 1.0251e+00],


In [7]:
batch.keys()

dict_keys(['input_features', 'attention_mask', 'binary_targets', 'regression_targets', 'regression_mask', 'file_name'])

# Model

In [ ]:
from model import WellTransformer
from train import train_model

In [32]:
model = WellTransformer(
    input_dim=3,
    d_model=256,
    nhead=8,
    num_encoder_layers=6
)

In [8]:
device='cuda'

In [9]:
batch = next(iter(train_loader))
input_features = batch['input_features'].to(device)
attention_mask = batch['attention_mask'].to(device)
binary_targets = batch['binary_targets'].to(device)
regression_targets = batch['regression_targets'].to(device)


In [33]:
model.to(device)
print('hi')

hi


In [10]:
# Forward pass
binary_output, regression_output = model(input_features, attention_mask)

In [16]:
batch['file_name']

['6792d33a-c5f1-414c-b0bd-8f8fff67554b',
 '85d034af-0a38-4588-a7d9-ad68d526323a',
 '63a564e6-b3b9-40f8-9431-308f00fe102a',
 'd0544fba-cb05-4bab-bec7-e44b50324356',
 '17ab789d-fe4e-4ef7-818f-92f13ee59cb7',
 '0d27476d-c62c-4cb1-9843-fc3e93f2671a',
 'f1a702ea-0b47-49ba-baf4-5ab5aee0e942',
 'b4f50901-5900-47e3-9fec-7f49933507df',
 '42a6052e-e0c7-4da6-96ad-c5c3af3ebf09',
 '60be794b-67ec-4cee-9aab-463784831920',
 'd1033674-5968-4878-bfab-46afe9b09054',
 'a887ed91-e50f-4f9f-a33a-0dddf08ebdd9',
 '8db0eafc-0330-4366-9dda-f11a997002af',
 '22e81afb-db50-4092-aea2-7bcef8a0ecbc',
 'b4eb3cd2-a437-493e-b95e-c7259afabe5b',
 'ef761615-02e5-4309-9f1e-6ad87b2d436d',
 '4a6c6668-ce49-4b75-87c6-242f1b46d636',
 '3c7456b4-5237-4956-9f68-11ebc0274088',
 '48a3d1ac-fefc-4267-83c0-0c300029213c',
 'c9daa53d-2deb-402a-bc7a-7dde765acdd9',
 'dadfe47e-6f35-4e19-969f-2518499146b2',
 '1f0da69a-bf38-4ec3-b315-ab0fe5c71102',
 '9d9c2e0f-945e-401e-aea6-d67fc0feddfe',
 'de4033ac-fd6c-4a5b-9dfb-39ae35b58bbe',
 '0216d406-0d17-

In [11]:
regression_targets

tensor([[ 1.4441e+00,  1.4013e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.5648e+00,  4.5170e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.7228e+00,  8.9881e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.1507e+02],
        [ 2.5305e+00,  1.3800e+00,  1.9359e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 3.8128e+00,  1.4936e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.6232e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.5425e+00,  0.0000e+00,  9.5508e-01,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 6.8915e-01,  0.0000e+00,  0.0000e+00,  9.0825e-01,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 

In [12]:
regression_output

tensor([[ 1.9815,  0.8004,  0.2070,  1.0424,  1.7658, 12.2642,  6.8711],
        [ 1.9603,  0.7808,  0.1896,  1.0431,  1.7648, 12.2769,  6.8593],
        [ 1.9726,  0.7968,  0.1965,  1.0500,  1.7646, 12.2660,  6.8733],
        [ 2.0045,  0.8196,  0.2113,  1.0546,  1.7668, 12.2489,  6.8757],
        [ 2.0036,  0.8202,  0.2031,  1.0640,  1.7640, 12.2489,  6.8754],
        [ 2.0024,  0.8126,  0.2048,  1.0547,  1.7679, 12.2526,  6.8630],
        [ 1.9397,  0.7620,  0.1829,  1.0373,  1.7674, 12.2828,  6.8561],
        [ 2.0028,  0.8208,  0.2143,  1.0545,  1.7728, 12.2414,  6.8703],
        [ 1.9657,  0.7936,  0.2023,  1.0407,  1.7581, 12.2640,  6.8708],
        [ 1.9770,  0.7975,  0.2042,  1.0424,  1.7656, 12.2675,  6.8698],
        [ 1.9810,  0.7927,  0.1919,  1.0557,  1.7725, 12.2692,  6.8631],
        [ 1.9323,  0.7581,  0.1826,  1.0310,  1.7636, 12.2848,  6.8505],
        [ 1.8785,  0.7646,  0.1975,  1.0162,  1.7486, 12.2433,  6.8492],
        [ 1.9487,  0.7767,  0.1924,  1.0375,  1.763

In [25]:
import torch

In [26]:
torch.__version__

'2.6.0+cu126'

In [13]:
binary_output

tensor([[0.1743, 0.7972, 0.5401, 0.2663, 0.2664, 0.1445, 0.1287, 0.1205],
        [0.1740, 0.7980, 0.5410, 0.2670, 0.2672, 0.1444, 0.1282, 0.1201],
        [0.1729, 0.7980, 0.5424, 0.2669, 0.2679, 0.1451, 0.1286, 0.1209],
        [0.1729, 0.7976, 0.5412, 0.2655, 0.2667, 0.1460, 0.1286, 0.1213],
        [0.1715, 0.7986, 0.5438, 0.2657, 0.2695, 0.1466, 0.1289, 0.1212],
        [0.1730, 0.8003, 0.5401, 0.2665, 0.2698, 0.1456, 0.1274, 0.1205],
        [0.1757, 0.7946, 0.5421, 0.2662, 0.2634, 0.1445, 0.1297, 0.1205],
        [0.1726, 0.7999, 0.5391, 0.2659, 0.2687, 0.1445, 0.1270, 0.1205],
        [0.1742, 0.7979, 0.5403, 0.2680, 0.2702, 0.1445, 0.1287, 0.1202],
        [0.1743, 0.7972, 0.5399, 0.2661, 0.2659, 0.1445, 0.1286, 0.1206],
        [0.1724, 0.7990, 0.5413, 0.2659, 0.2666, 0.1450, 0.1278, 0.1204],
        [0.1774, 0.7936, 0.5412, 0.2671, 0.2638, 0.1441, 0.1297, 0.1203],
        [0.1795, 0.7940, 0.5375, 0.2690, 0.2638, 0.1414, 0.1303, 0.1200],
        [0.1759, 0.7943, 0.5408, 0.267

In [36]:
regression_output

tensor([[-0.0799,  0.1651,  0.0431, -0.2534,  0.2002, -0.0372,  0.4909],
        [-0.2676,  0.0895, -0.0709, -0.1633,  0.0310,  0.0316,  0.4695],
        [-0.0647, -0.0453, -0.0756, -0.1928,  0.0414,  0.3822,  0.4602],
        [-0.5158,  0.1917, -0.0062, -0.5409, -0.0301,  0.2594,  0.3123],
        [-0.0967,  0.0231,  0.0659, -0.2884, -0.0229,  0.3029,  0.3925],
        [ 0.0862, -0.0982,  0.1590,  0.0287,  0.1546,  0.1567,  0.2391],
        [ 0.0527, -0.2035,  0.1338, -0.2445, -0.1296,  0.2083,  0.2712],
        [-0.5058,  0.1675, -0.0697, -0.3646,  0.0571,  0.0802,  0.4114],
        [-0.1591, -0.0722,  0.0478, -0.4445,  0.1089,  0.1614,  0.5998],
        [-0.1494,  0.1073,  0.1564, -0.4769,  0.0034,  0.2700,  0.5165],
        [-0.2837,  0.2268,  0.1116, -0.1756, -0.0866,  0.0503,  0.3421],
        [-0.3557,  0.0136, -0.0009, -0.3917, -0.0776,  0.2966,  0.5447],
        [-0.1480,  0.0083, -0.1019, -0.2632, -0.1213,  0.3040,  0.4917],
        [-0.4308,  0.1718,  0.0487, -0.3503, -0.079

# Train

In [17]:
# Create data loaders
train_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_r_sc.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

val_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_r_sc.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

# Create model
model = TransformerModel(
    input_dim=3,
    d_model=64,
    nhead=4,
    num_encoder_layers=2,
    dim_feedforward = 256,
    dropout = 0.1,
    max_seq_len = 1024
)



Preprocessing time series data...


100%|█████████████████████████████████████████████████████████████████████████| 44884/44884 [00:03<00:00, 13853.03it/s]


Preprocessed 1000 time series
Preprocessing time series data...


100%|█████████████████████████████████████████████████████████████████████████| 44884/44884 [00:03<00:00, 14199.48it/s]


Preprocessed 1000 time series


In [3]:
# Create data loaders
train_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_r_sc.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

val_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_r_sc.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

Preprocessing time series data...


100%|█████████████████████████████████████████████████████████████████████████| 44884/44884 [00:03<00:00, 13517.57it/s]


Preprocessed 1000 time series
Preprocessing time series data...


100%|█████████████████████████████████████████████████████████████████████████| 44884/44884 [00:03<00:00, 13856.38it/s]


Preprocessed 1000 time series


In [3]:
# Create data loaders
train_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f_adv.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_r_sc_adv.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

val_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f_adv.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_r_sc_adv.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

Preprocessing time series data...


100%|█████████████████████████████████████████████████████████████████████████| 44809/44809 [00:03<00:00, 13603.71it/s]


Preprocessed 1000 time series
Preprocessing time series data...


100%|█████████████████████████████████████████████████████████████████████████| 44809/44809 [00:03<00:00, 13806.40it/s]


Preprocessed 1000 time series


In [8]:
# Create model
model = TransformerModel_ver2(
    input_dim=10,
    d_model=256,
    nhead=4,
    num_encoder_layers=4,
    dim_feedforward=512,
    dropout=0.1,
    max_seq_len = 1024
)

In [ ]:
Train Loss: 2.1410
Val Loss: 2.0866

In [9]:
# Train model
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=2,
    learning_rate=1e-4,
    device='cuda',
    binary_loss_weight=1.0,
    regression_loss_weight=1.0,
    pos_weight=1.0
)


Epoch 1/2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00,  8.95it/s]


Train Loss: 1.8111, Binary Loss: 0.6187, Regression Loss: 1.1924
Val Loss: 1.5627, Binary Loss: 0.5248, Regression Loss: 1.0379, AUROC: 0.5164, MSE: 0.3388, Mean Accuracy: 0.5518
New best model saved with validation mean accuracy: 0.5518

Epoch 2/2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00,  8.91it/s]


Train Loss: 1.5040, Binary Loss: 0.5182, Regression Loss: 0.9858
Val Loss: 1.4775, Binary Loss: 0.5145, Regression Loss: 0.9630, AUROC: 0.5226, MSE: 0.3204, Mean Accuracy: 0.5550
New best model saved with validation mean accuracy: 0.5550


In [11]:
# Train model
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=2,
    learning_rate=1e-4,
    device='cuda',
    binary_loss_weight=1.0,
    regression_loss_weight=1.0,
    pos_weight=1.0
)


Epoch 1/2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 11.06it/s]


Train Loss: 2.0473, Binary Loss: 0.6422, Regression Loss: 1.4050
Val Loss: 1.7450, Binary Loss: 0.5791, Regression Loss: 1.1659, AUROC: 0.4670, MSE: 0.3797, Mean Accuracy: 0.5200
New best model saved with validation mean accuracy: 0.5200

Epoch 2/2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 11.01it/s]


Train Loss: 1.7052, Binary Loss: 0.5678, Regression Loss: 1.1374
Val Loss: 1.6863, Binary Loss: 0.5586, Regression Loss: 1.1277, AUROC: 0.4769, MSE: 0.3645, Mean Accuracy: 0.5248
New best model saved with validation mean accuracy: 0.5248


In [18]:
# Train model
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=2,
    learning_rate=1e-4,
    device='cuda',
    binary_loss_weight=1.0,
    regression_loss_weight=1.0,
    pos_weight=2.0
)


Epoch 1/2


Training: 100%|█████████████████████| 16/16 [00:01<00:00,  9.78it/s, loss=2.34, binary_loss=0.89, regression_loss=1.45]
C:\Users\vorsi\miniconda3\envs\and_kar_c\lib\site-packages\torch\nn\modules\transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\NestedTensorImpl.cpp:182.)
  output = torch._nested_tensor_from_mask(
Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 64.52it/s]


Train Loss: 2.3649, Binary Loss: 0.8921, Regression Loss: 1.4728
Val Loss: 2.1437, Binary Loss: 0.8643, Regression Loss: 1.2794, AUROC: 0.5080, MSE: 0.4092, Mean Accuracy: 0.5700
New best model saved with validation loss: 2.1437

Epoch 2/2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 79.27it/s]


Train Loss: 2.0861, Binary Loss: 0.8544, Regression Loss: 1.2317
Val Loss: 2.0252, Binary Loss: 0.8482, Regression Loss: 1.1769, AUROC: 0.5070, MSE: 0.3820, Mean Accuracy: 0.6001
New best model saved with validation loss: 2.0252


# CV parts

In [8]:
from cv_dataloader import create_image_data_loader, TimeSeriesImageDataset, TimeSeriesImageInferenceDataset

#from cv_model import ViTTimeSeriesModel, ViTTimeSeriesModel_v2
from cv_model import ViTTimeSeriesModel, ViTTimeSeriesModel_v2, create_model
from cv_train import train_cv_model, inference_cv, save_cv_predictions
from cv_example import plot_training_history
#from train import save_predictions

In [8]:
dataset = TimeSeriesImageDataset(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f_adv.csv',
    images_dir='E:/DS/data/siam/plots/deb/',
)

In [9]:
b = next(iter(dataset))

In [10]:
b.keys()

dict_keys(['image', 'binary_targets', 'regression_targets', 'regression_masks', 'file_name'])

In [13]:
loader = create_image_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f_adv.csv',
    images_dir='E:/DS/data/siam/plots/deb/',
    batch_size = 4,
    shuffle = True,
    num_workers = 0
)

In [14]:
batch = next(iter(loader))

In [15]:
batch.keys()

dict_keys(['image', 'binary_targets', 'regression_targets', 'regression_masks', 'file_name'])

In [2]:
train_loader = create_image_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f_adv.csv',
    images_dir='E:/DS/data/siam/plots/deb/',
    batch_size = 32,
    shuffle = True,
    num_workers = 0
)

In [3]:
val_loader = create_image_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/deb1k_df_woo_r_sc_f_adv.csv',
    images_dir='E:/DS/data/siam/plots/deb/',
    batch_size = 32,
    shuffle = False,
    num_workers = 0
)

In [4]:
test_loader = create_image_data_loader(
    csv_path=None,
    images_dir='E:/DS/data/siam/plots/deb/',
    batch_size = 32,
    shuffle = False,
    num_workers = 0
)

In [6]:
model = create_model(
    model_type = "vit",
    #model_name='b7',
    n_binary_targets = 8,
    n_regression_targets = 7,
    dropout = 0.1,
    freeze_backbone = False,
    use_multi_layer_features = False,
    pretrained = True
)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
trained_model, history = train_cv_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=2,  # Adjust as needed
    learning_rate=2e-5,  # Lower learning rate for pretrained model
    device='cuda',
    binary_loss_weight=1.0,
    regression_loss_weight=1.,
    pos_weight=1.,
    warmup_epochs=1
)


Epoch 1/2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 32/32 [00:19<00:00,  1.60it/s]


Train Loss: 2.1247, Binary Loss: 0.6297, Regression Loss: 1.4949
Val Loss: 1.8931, Binary Loss: 0.4935, Regression Loss: 1.3996, AUROC: 0.5381, MSE: 0.4426, Mean Accuracy: 0.5646
New best model saved with validation mean accuracy: 0.5646

Epoch 2/2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 32/32 [00:19<00:00,  1.62it/s]


Train Loss: 1.7475, Binary Loss: 0.4817, Regression Loss: 1.2659
Val Loss: 1.6466, Binary Loss: 0.4687, Regression Loss: 1.1779, AUROC: 0.6234, MSE: 0.3726, Mean Accuracy: 0.5694
New best model saved with validation mean accuracy: 0.5694


In [9]:
res =inference_cv(model = trained_model, dataloader=test_loader,)

Running inference: 100%|███████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.53it/s]


In [35]:
plot_training_history(history)